In [1]:
!pip install langchain --target /home/aah9103/.local/lib/python3.11/site-packages

  Using cached langchain-0.1.16-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.29-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.34-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_core-0.1.46-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached langsmith-0.1.51-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using

In [2]:
!pip install pdf2image --target /home/aah9103/.local/lib/python3.11/site-packages

  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pillow-10.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Using cached pillow-10.3.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)


In [3]:
!pip install GPT4All --target /home/aah9103/.local/lib/python3.11/site-packages

  Using cached gpt4all-2.6.0-py3-none-manylinux1_x86_64.whl.metadata (4.1 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
Using cached gpt4all-2.6.0-py3-none-manylinux1_x86_64.whl (3.9 MB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (140 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
ERROR: pip's dependency resolver does not currently take

In [4]:
import sys
sys.executable


'/share/apps/anaconda3/2024.02/bin/python'

In [5]:
sys.path

['/home/aah9103',
 '/share/apps/anaconda3/2024.02/lib/python311.zip',
 '/share/apps/anaconda3/2024.02/lib/python3.11',
 '/share/apps/anaconda3/2024.02/lib/python3.11/lib-dynload',
 '',
 '/home/aah9103/.local/lib/python3.11/site-packages',
 '/share/apps/anaconda3/2024.02/lib/python3.11/site-packages']

In [1]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
import re
import random
import pandas as pd
import json

In [7]:
# #Load dictionary:
# data = {0: {'top 5 sample': ['water', 'area', 'river', 'park', 'miles'], 'intruder': 'game'}, 
#         1: {'top 5 sample': ['horses', 'horse', 'breed', 'coins', 'silver'], 'intruder': 'hindu'},
#         2: {'top 5 sample': ['issue', 'situation', 'lead', 'stick', 'nice'], 'intruder': 'tampon'}}

# #gensim VI is worse than mallet GS 
# #10-20-50 topics. regenerate 20 times(mutlitple runs) and then over multiple datasetsg

In [2]:
#Load Gensim File
dataG = pd.read_csv('gensim_test.csv')
dataG['package'] = 'gensim'
#dataG.head()

In [4]:
#Load Mallet file
dataM = pd.read_csv('mallet_test.csv')
dataM['package'] = 'mallet'
#dataM.head()

In [5]:
#Shorten for test
dataM = dataM.head(1)
dataG = dataG.head(1)

In [6]:
#data = pd.concat([dataG, dataM], ignore_index=True)#.reset_index()
data = dataG
data.head(50)

,num_topics,seed,score,alpha,eta,tests,package
0,10,1928,0.615142,asymmetric,0.01,"{0: {'top 5 sample': ['start', 'time', 'feel',...",gensim


In [24]:
# #load json output of topic model
# with open('llm_tests.json', 'r') as f:
#     data = json.load(f)

# f.close()


In [7]:
# Function to fix JSON strings and convert them to JSON objects
def convertJSON(s):
    s = s.replace("'", '"')
    s = re.sub(r'(\s*)(\d+)(\s*):', r'\1"\2"\3:', s)
    return json.loads(s)

In [8]:
#Load json string
data['tests'] = data['tests'].apply(lambda x: convertJSON(x))

In [11]:
# #load json output of topic model
# with open('gensim_tests.json', 'r') as f:
#     data = json.load(f)

# f.close()


In [10]:
llm = GPT4All(model="./models/gptj/mistral-7b-openorca.gguf2.Q4_0.gguf", backend = 'gptj', verbose = False)

In [11]:
template = """
You are a helpful assistant evaluating the top words of a topic model output for a given topic. 
Select which word or phrase is the least related to all other words or phrases in terms of the meaning of the word or phrase. 
If multiple words do not fit, choose the word that is most out of place in terms of the meaning of the word.
{datasetDescription}
Here is the list of words for your task: [{listOfWords}]
Your answer MUST CONTAIN ONLY a single word.
Also, mark the selected word by preceding it with 000 and succeeding it with 000.
For example if a list of words is "apple, banana, cat", your response should ONLY BE: '000cat000'

"""

In [12]:
prompt = PromptTemplate(template = template, input_variables = ['datasetDescription', 'listOfWords'])

In [13]:
llmChain = LLMChain(prompt = prompt, llm = llm, verbose = False)

In [16]:
# ans = llmChain({'datasetDescription': 'data of words', 'listOfWords': 'apple, banana, cat, sheep, house'})

# regSearch = re.search(r'000(.*?)000', ans['text'])
# if regSearch:
#     print(regSearch.group(1))
# else:
#     print('no match')

# You are a helpful assistant evaluating the top words of a topic model output for a given topic. 
# Select which word or phrase is the least related to all other words or phrases in terms of the meaning of the word or phrase. 
# If multiple words do not fit, choose the word that is most out of place in terms of the meaning of the word.
# Pay attention to the similarity of each word to all othe words in the set when you make your decision.
# Here is the list of words for your task: [hour, gain, week, baby, labor, section]
# Your answer MUST CONTAIN ONLY a single word.
# Also, mark the selected word by preceding it with 000 and succeeding it with 000.
# For example if a list of words is "apple, banana, cat", your response should ONLY BE: '000cat000'


In [14]:
dataDesc = """
Pay attention to the similarity of each word to all othe words in the set when you make your decision.
"""

# dataDesc = """
# The topic modelling is based on a corpus of reddit posts relating to maternal health. 
# The data from which topic words are extracted includes topics like abortion and sexuality.
# """

In [ ]:
# for k, v in data.items():
#     wordList = v['top 5 sample'] + [v['intruder']]
#     random.shuffle(wordList)
#     wordListStr = ", ".join(wordList)
#     trueIntruder = v['intruder']
#     detectedIntruder = 'NOTDETECTED'
#     ans = llmChain({'datasetDescription': dataDesc, 'listOfWords': wordListStr})
#     regSearch = re.search(r'000(.*?)000', ans['text'])
#     data[k]['wordlist'] = wordList
#     data[k]['wordliststr'] = wordListStr
#     if regSearch:
#         detectedIntruder = regSearch.group(1)
#     data[k]['detectedIntruder'] = detectedIntruder
#     if trueIntruder.lower() == detectedIntruder.lower():
#         data[k]['success'] = 1
#     else:
#         data[k]['success'] = 0
 

In [ ]:
# data

In [ ]:
# df = pd.DataFrame.from_dict(data, orient='index')
# rate = df['success'].sum() / len(df)
# rate

In [15]:
def llmEval(data, llmChain, maxIter = 10, dataDesc = """"""):
    tot = len(data)
    i = 1
    for k, v in data.items():
        print("Evaluating ",i," / ",tot)
        i += 1
        itr = maxIter
        wordList = v['top 5 sample'] + [v['intruder']]
        random.shuffle(wordList)
        wordListStr = ", ".join(wordList)
        trueIntruder = v['intruder']
        data[k]['wordlist'] = wordList
        data[k]['wordliststr'] = wordListStr
        detectedIntruder = 'NOTDETECTED'
        while itr > 0:
            itr -= 1
            if detectedIntruder != 'NOTDETECTED':
                itr = 0
                break
            ans = llmChain({'datasetDescription': dataDesc, 'listOfWords': wordListStr})
            regSearch = re.search(r'000(.*?)000', ans['text'])
            if regSearch:
                detectedIntruder = regSearch.group(1)

        data[k]['detectedIntruder'] = detectedIntruder
        data[k]['rawLLMOP'] = ans['text']
        if trueIntruder.lower() == detectedIntruder.lower():
            data[k]['success'] = 1
        else:
            data[k]['success'] = 0

    df = pd.DataFrame.from_dict(data, orient='index')
    rate = df['success'].sum() / len(df)
    return pd.Series({'score': rate, 'data': data})



In [16]:
#Run eval
data[['score', 'data']] = data['tests'].apply(lambda x: llmEval(x, llmChain, 10, dataDesc))

Evaluating  1  /  10


C:\ProgramData\Anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Evaluating  2  /  10
Evaluating  3  /  10
Evaluating  4  /  10
Evaluating  5  /  10
Evaluating  6  /  10
Evaluating  7  /  10
Evaluating  8  /  10
Evaluating  9  /  10
Evaluating  10  /  10


In [20]:
data

,num_topics,seed,score,alpha,eta,tests,package,data,NewScore,fixedData
0,10,1928,0.5,asymmetric,0.01,"{'0': {'top 5 sample': ['start', 'time', 'feel...",gensim,"{'0': {'top 5 sample': ['start', 'time', 'feel...",0.5,"{'0': {'top 5 sample': ['start', 'time', 'feel..."


In [18]:
def fixEval(data):
    newData = pd.DataFrame.from_dict(data, orient= 'index')[['top 5 sample', 'wordliststr', 'rawLLMOP', 'intruder', 'detectedIntruder', 'success']]
    newData = newData.to_dict(orient='index')
    tot = len(data)
    i = 1
    for k, v in newData.items():
        i += 1
        trueIntruder = newData[k]['intruder']
        detectedIntruder = newData[k]['detectedIntruder']
        llmOP = newData[k]['rawLLMOP']
        # llmOP = 'is: int'
        # trueIntruder = 'test'
        # detectedIntruder = 'NOTDETECTED'
#        print(trueIntruder)
#        detectedIntruder = 'NOTDETECTED'
        if detectedIntruder == 'NOTDETECTED':
            print("fixing ",i," / ",tot)
            regSearch = re.search(r"is:\W*([a-zA-Z]+)\W*", llmOP)
            if regSearch:
                detectedIntruder = regSearch.group(1)
#        print(detectedIntruder)

        newData[k]['detectedIntruder'] = detectedIntruder
        if trueIntruder.lower() == detectedIntruder.lower():
            newData[k]['success'] = 1
        else:
            newData[k]['success'] = 0

    df = pd.DataFrame.from_dict(newData, orient= 'index')[['top 5 sample', 'wordliststr', 'rawLLMOP', 'intruder', 'detectedIntruder', 'success']]
    rate = df['success'].sum() / len(df)
    return pd.Series({'NewScore': rate, 'fixedData': newData})



In [94]:
#databkp = data
#data = databkp

In [19]:
#Run repair
data[['NewScore', 'fixedData']] = data['data'].apply(lambda x: fixEval(x))

In [37]:
#dcols = list(data.columns)
#dcols.remove(['tests', 'data'])
#data[['num_topics', 'seed', 'score', 'alpha', 'eta', 'package']]
#dcols

,num_topics,seed,score,alpha,eta,package
0,10,1928,0.50,asymmetric,0.01,gensim
1,10,3328,0.10,10,0.01,gensim
2,10,1062,0.40,10,NaN,gensim
3,10,3953,0.10,10,0.01,gensim
4,10,1172,0.20,10,NaN,gensim
5,20,1928,0.15,asymmetric,0.10,gensim
6,20,3328,0.35,0.01,NaN,gensim
7,20,1062,0.15,10,NaN,gensim
8,20,3953,0.20,asymmetric,0.01,gensim
9,20,1172,0.50,10,0.10,gensim


In [117]:
data.to_excel('evalOutputGensim2.xlsx')#, mode='a', header=False)
#data[['num_topics', 'seed', 'score', 'alpha', 'eta', 'package']].to_csv('evalOutputGensim1.csv')
# pd.DataFrame.from_dict(dict(data.head(1)['data'])[0], orient= 'index')[['top 5 sample', 'wordliststr', 'rawLLMOP', 'intruder', 'detectedIntruder', 'success']]

In [ ]:
data.to_csv('evalOutput1.csv')
# pd.DataFrame.from_dict(dict(data.head(1)['data'])[0], orient= 'index')[['top 5 sample', 'wordliststr', 'rawLLMOP', 'intruder', 'detectedIntruder', 'success']]

In [109]:
data.head(50)

,num_topics,seed,score,alpha,eta,tests,package,data,NewScore,fixedData
0,10,1928,0.50,asymmetric,0.01,"{'0': {'top 5 sample': ['start', 'time', 'feel', 'back', 'make'], 'intruder': 'baby', 'wordlist': ['back', 'make', 'feel', 'time', 'start', 'baby'], 'wordliststr': 'back, make, feel, time, start, baby', 'detectedIntruder': 'baby', 'rawLLMOP': 'The least related word in this set is: 000baby000', 'success': 1}, '1': {'top 5 sample': ['blood', 'sex', 'doctor', 'problem', 'year'], 'intruder': 'week', 'wordlist': ['year', 'problem', 'doctor', 'sex', 'blood', 'week'], 'wordliststr': 'year, problem, doctor, sex, blood, week', 'detectedIntruder': 'sex', 'rawLLMOP': 'The least related word in this set is: 000sex000', 'success': 0}, '2': {'top 5 sample': ['human', 'life', 'abortion', 'make', 'child'], 'intruder': 'week', 'wordlist': ['make', 'abortion', 'human', 'week', 'life', 'child'], 'wordliststr': 'make, abortion, human, week, life, child', 'detectedIntruder': 'week', 'rawLLMOP': 'The least related word in this set is: 000week000', 'success': 1}, '3': {'top 5 sample': ['hour', 'labor', ...",gensim,"{'0': {'top 5 sample': ['start', 'time', 'feel', 'back', 'make'], 'intruder': 'baby', 'wordlist': ['back', 'make', 'feel', 'time', 'start', 'baby'], 'wordliststr': 'back, make, feel, time, start, baby', 'detectedIntruder': 'baby', 'rawLLMOP': 'The least related word in this set is: 000baby000', 'success': 1}, '1': {'top 5 sample': ['blood', 'sex', 'doctor', 'problem', 'year'], 'intruder': 'week', 'wordlist': ['year', 'problem', 'doctor', 'sex', 'blood', 'week'], 'wordliststr': 'year, problem, doctor, sex, blood, week', 'detectedIntruder': 'sex', 'rawLLMOP': 'The least related word in this set is: 000sex000', 'success': 0}, '2': {'top 5 sample': ['human', 'life', 'abortion', 'make', 'child'], 'intruder': 'week', 'wordlist': ['make', 'abortion', 'human', 'week', 'life', 'child'], 'wordliststr': 'make, abortion, human, week, life, child', 'detectedIntruder': 'week', 'rawLLMOP': 'The least related word in this set is: 000week000', 'success': 1}, '3': {'top 5 sample': ['hour', 'labor', ...",0.50,"{'0': {'top 5 sample': ['start', 'time', 'feel', 'back', 'make'], 'wordliststr': 'back, make, feel, time, start, baby', 'rawLLMOP': 'The least related word in this set is: 000baby000', 'intruder': 'baby', 'detectedIntruder': 'baby', 'success': 1}, '1': {'top 5 sample': ['blood', 'sex', 'doctor', 'problem', 'year'], 'wordliststr': 'year, problem, doctor, sex, blood, week', 'rawLLMOP': 'The least related word in this set is: 000sex000', 'intruder': 'week', 'detectedIntruder': 'sex', 'success': 0}, '2': {'top 5 sample': ['human', 'life', 'abortion', 'make', 'child'], 'wordliststr': 'make, abortion, human, week, life, child', 'rawLLMOP': 'The least related word in this set is: 000week000', 'intruder': 'week', 'detectedIntruder': 'week', 'success': 1}, '3': {'top 5 sample': ['hour', 'labor', 'birth', 'push', 'hospital'], 'wordliststr': 'week, hour, hospital, birth, labor, push', 'rawLLMOP': 'The least related word in this set is: 000hospital000', 'intruder': 'week', 'detectedIntruder': ..."
1,10,3328,0.10,10,0.01,"{'0': {'top 5 sample': ['iud', 'month', 'bad', 'heavy', 'period'], 'intruder': 'make', 'wordlist': ['iud', 'bad', 'heavy', 'month', 'make', 'period'], 'wordliststr': 'iud, bad, heavy, month, make, period', 'detectedIntruder': 'iud', 'rawLLMOP': 'The least related word in this set is: 000iud000', 'success': 0}, '1': {'top 5 sample': ['give', 'time', 'thing', 'hard', 'baby'], 'intruder': 'period', 'wordlist': ['time', 'give', 'thing', 'baby', 'hard', 'period'], 'wordliststr': 'time, give, thing, baby, hard, period', 'detectedIntruder': 'baby', 'rawLLMOP': 'The least related word in this set is: 000baby000', 'success': 0}, '2': {'top 5 sample': ['time', 'pain', 'cervix', 'check', 'doctor'], 'intruder': 'day', 'wordlist': ['check', 'cervix', 'day', 'pain', 'time', 'doctor'], 'wordliststr': 'check, cervix, day, pain, time, doctor', 'detectedIntruder

In [47]:
pd.DataFrame.from_dict(dict(data.tail(1)['data'])[19], orient= 'index')[['top 5 sample', 'wordliststr', 'rawLLMOP', 'intruder', 'detectedIntruder', 'success']]

,top 5 sample,wordliststr,rawLLMOP,intruder,detectedIntruder,success
0,"[horrendous, kilogram, lange, endometriosis, shield]","horrendous, shield, kilogram, endometriosis, mammogram, lange",The least related word in this set is: 000kilogram000,mammogram,kilogram,0
1,"[horrendous, kilogram, lange, endometriosis, stretch_mark]","mammogram, stretch_mark, horrendous, endometriosis, lange, kilogram",The least related word in this set is: 000lange000,mammogram,lange,0
2,"[horrendous, kilogram, muggle_blood, lange, arthritis]","kilogram, coat_factory, horrendous, arthritis, muggle_blood, lange",The least related word in this set is: 000coat_factory000,coat_factory,coat_factory,1
3,"[tad, surreal, anxious, kilogram, endometriosis]","compute, anxious, endometriosis, kilogram, surreal, tad",The least related word in this set is: kilogram,compute,NOTDETECTED,0
4,"[horrendous, kilogram, lange, endometriosis, shield]","horrendous, shield, kilogram, mammogram, endometriosis, lange",The least related word in this set is: 000kilogram000,mammogram,kilogram,0
5,"[horrendous, kilogram, lange, ozzydog, shield]","ozzydog, lange, mammogram, shield, kilogram, horrendous",The least related word in this set is: 000horrendous000,mammogram,horrendous,0
6,"[horrendous, kilogram, lange, endometriosis, shield]","horrendous, mammogram, endometriosis, kilogram, lange, shield",The least related word in this set is: 000lange000,mammogram,lange,0
7,"[horrendous, kilogram, lange, ozzydog, shield]","lange, kilogram, shield, ozzydog, horrendous, mammogram",The least related word in this set is: 000ozzydog000,mammogram,ozzydog,0
8,"[horrendous, kilogram, lange, ozzydog, shield]","lange, kilogram, horrendous, mammogram, shield, ozzydog",The least related word in this set is: 000ozzydog000,mammogram,ozzydog,0
9,"[horrendous, kilogram, lange, endometriosis, shield]","shield, horrendous, mammogram, lange, endometriosis, kilogram",The least related word in this set is: 000kilogram000,mammogram,kilogram,0


In [ ]:
# #Without dataset description
# dataDesc = """"""
# dataDesc = """
# This is a dataset of words
# """
# # dataDesc = """
# # The topic modelling is based on a corpus of reddit posts relating to maternal health. 
# # The data from which topics are extracted includes topics like abortion and sexuality.
# # """

# for k, v in data.items():
#     wordList = v['top 5 sample'] + [v['intruder']]
#     random.shuffle(wordList)
#     wordListStr = ", ".join(wordList)
#     trueIntruder = v['intruder']
#     detectedIntruder = 'NOTDETECTED'
#     ans = llmChain({'datasetDescription': dataDesc, 'listOfWords': wordListStr})
#     regSearch = re.search(r'000(.*?)000', ans['text'])
#     data[k]['wordlist'] = wordList
#     data[k]['wordliststr'] = wordListStr
#     if regSearch:
#         detectedIntruder = regSearch.group(1)
#     data[k]['detectedIntruder'] = detectedIntruder
#     data[k]['rawLLMOP'] = ans['text']
#     if trueIntruder.lower() == detectedIntruder.lower():
#         data[k]['success'] = 1
#     else:
#         data[k]['success'] = 0

# df = pd.DataFrame.from_dict(data, orient='index')
# rate = df['success'].sum() / len(df)
# rate

In [ ]:
# data
##pd.DataFrame.from_dict(dict(data.head(1)['data'])[0], orient= 'index')
#dict(data.head(1)[['data']])

In [ ]:
# pd.DataFrame.from_dict(data, orient= 'index')[['top 5 sample', 'wordliststr', 'rawLLMOP', 'intruder', 'detectedIntruder', 'success']]

In [46]:
pd.set_option(
    'display.max_colwidth', 1000
)